# Attempt to realize a sequential Monte Carlo simulation (in the first suboptimal regime) supported by OpenMM 

*the first suboptimal regime implies particle weights that respect Eq. 31 in Del Moral, et. al. Sequential Monte Carlo Samplers 

In [1]:
import pickle
import numpy as np
from pkg_resources import resource_filename
from coddiwomple.openmm.coddiwomple import endstate_equilibration
from perses.annihilation.lambda_protocol import RelativeAlchemicalState
import os
from simtk import unit
import mdtraj as md
from coddiwomple.openmm.states import OpenMMPDFState, OpenMMParticleState
from coddiwomple.distribution_factories import TargetFactory, ProposalFactory
from coddiwomple.openmm.integrators import OMMLI
from coddiwomple.openmm.propagators import OMMBIP
from coddiwomple.particles import Particle
from coddiwomple.resamplers import MultinomialResampler
from coddiwomple.utils import nESS
from copy import deepcopy


RDKit WARNING: [16:15:15] Enabling RDKit 2019.09.3 jupyter extensions


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU


In [2]:
#load the factory to retrieve the system
factory_pickle = f"/data/perses_data/benzene_fluorobenzene.vacuum.factory.pkl"
_pickl = resource_filename('coddiwomple', factory_pickle)
with open(_pickl, 'rb') as f:
    factory = pickle.load(f)

#load the endstate cache
endstate_cache = md.Trajectory.load(f"tester/eq.0000.pdb")

#make a pdf state
pdf_state = OpenMMPDFState(system = factory.hybrid_system, pressure=None, alchemical_composability = RelativeAlchemicalState)

DEBUG:openmm_states:successfully instantiated OpenMMPDFState equipped with the following parameters: {'lambda_electrostatics_insert': 0.0, 'lambda_electrostatics': None, 'lambda_angles': 0.0, 'lambda_sterics_core': 0.0, 'lambda_torsions': 0.0, 'lambda_electrostatics_core': 0.0, 'lambda_sterics_delete': 0.0, 'lambda_electrostatics_delete': 0.0, 'lambda_bonds': 0.0, 'lambda_sterics': None, 'lambda_sterics_insert': 0.0}


conducting subsequent work with the following platform: CPU


In [3]:
default_functions = {'lambda_sterics_core':
                     lambda x: x,
                     'lambda_electrostatics_core':
                     lambda x: x,
                     'lambda_sterics_insert':
                     lambda x: 2.0 * x if x < 0.5 else 1.0,
                     'lambda_sterics_delete':
                     lambda x: 0.0 if x < 0.5 else 2.0 * (x - 0.5),
                     'lambda_electrostatics_insert':
                     lambda x: 0.0 if x < 0.5 else 2.0 * (x - 0.5),
                     'lambda_electrostatics_delete':
                     lambda x: 2.0 * x if x < 0.5 else 1.0,
                     'lambda_bonds':
                     lambda x: x,
                     'lambda_angles':
                     lambda x: x,
                     'lambda_torsions':
                     lambda x: x
                     }

lambda_sequence = [{key: default_functions[key](q) for key in default_functions.keys()} for q in np.linspace(0,1,10)]

In [4]:
#make a proposal factory and target factory
integrator = OMMLI()
propagator = OMMBIP(pdf_state, integrator)
proposal_factory = ProposalFactory(lambda_sequence, propagator)
target_factory = TargetFactory(OpenMMPDFState(system = factory.hybrid_system, pressure=None, alchemical_composability = RelativeAlchemicalState), lambda_sequence, termination_parameters = None)


DEBUG:openmm_propagators:initializing OMMLI...
DEBUG:openmm_propagators:OMMLI: metropolization is False
DEBUG:openmm_propagators:OMMLI: successfully parsed splitting string
DEBUG:openmm_propagators:OMMLI: adding global variables...
DEBUG:openmm_propagators:OMMLI: adding integrator steps...
DEBUG:openmm_propagators:OMMLI: adding substep functions...
DEBUG:openmm_propagators:successfully executed ABCMeta init.
DEBUG:openmm_propagators:successfully equipped integrator: OMMLI
DEBUG:openmm_propagators:integrator printable: None
DEBUG:distribution_factories:equipped pdf state and parameter sequence
DEBUG:distribution_factories:successfully equipped propagator: <coddiwomple.openmm.propagators.OMMBIP object at 0x7f5f0d849b00>


step      0 : allow forces to update the context state
step      1 : if(has_kT_changed = 1):
step      2 :    sigma <- sqrt(kT/m)
step      3 :    has_kT_changed <- 0
step      4 : end
step      5 : old_ke <- sum(0.5 * m * v * v)
step      6 : v <- v + (dt / 2) * f / m
step      7 : constrain velocities
step      8 : new_ke <- sum(0.5 * m * v * v)
step      9 : shadow_work <- shadow_work + (new_ke - old_ke)
step     10 : old_pe <- energy
step     11 : old_ke <- sum(0.5 * m * v * v)
step     12 : x <- x + ((dt / 2) * v)
step     13 : x1 <- x
step     14 : constrain positions
step     15 : v <- v + ((x - x1) / (dt / 2))
step     16 : constrain velocities
step     17 : new_pe <- energy
step     18 : new_ke <- sum(0.5 * m * v * v)
step     19 : shadow_work <- shadow_work + (new_ke + new_pe) - (old_ke + old_pe)
step     20 : old_ke <- sum(0.5 * m * v * v)
step     21 : v <- (a * v) + (b * sigma * gaussian)
step     22 : constrain velocities
step     23 : new_ke <- sum(0.5 * m * v * v)
step 

DEBUG:openmm_states:successfully instantiated OpenMMPDFState equipped with the following parameters: {'lambda_electrostatics_insert': 0.0, 'lambda_electrostatics': None, 'lambda_angles': 0.0, 'lambda_sterics_core': 0.0, 'lambda_torsions': 0.0, 'lambda_electrostatics_core': 0.0, 'lambda_sterics_delete': 0.0, 'lambda_electrostatics_delete': 0.0, 'lambda_bonds': 0.0, 'lambda_sterics': None, 'lambda_sterics_insert': 0.0}


conducting subsequent work with the following platform: CPU


DEBUG:distribution_factories:equipped pdf state and parameter sequence
DEBUG:distribution_factories:set termination parameters as: {'lambda_sterics_core': 1.0, 'lambda_electrostatics_core': 1.0, 'lambda_sterics_insert': 1.0, 'lambda_sterics_delete': 1.0, 'lambda_electrostatics_insert': 1.0, 'lambda_electrostatics_delete': 1.0, 'lambda_bonds': 1.0, 'lambda_angles': 1.0, 'lambda_torsions': 1.0}


In [5]:
#make a multinomial resampler
observable = nESS
def threshold_function(_observable, **kwargs):
    returnable = False if _observable > 0.5 else True
    return returnable
    
resampler = MultinomialResampler()

DEBUG:resamplers:instantiating empty resampling logger


In [6]:
#create particles and equip initial particle states
num_particles = 3
particles = [Particle(index = idx) for idx in range(num_particles)]

n_frames = endstate_cache.n_frames

for idx in range(num_particles):
    random_frame = np.random.choice(range(n_frames))
    positions = endstate_cache.xyz[random_frame] * unit.nanometers
    box_vectors = endstate_cache.unitcell_vectors[random_frame]*unit.nanometers
    particle_state = OpenMMParticleState(positions=positions, box_vectors = box_vectors)
    proposal_factory.equip_initial_sample(particle = particles[idx], 
                                          initial_particle_state = particle_state, 
                                          generation_pdf = None)


DEBUG:openmm_states:successfully updated OpenMMPDFState parameters as follows: {'lambda_sterics_core': 0.0, 'lambda_electrostatics_core': 0.0, 'lambda_sterics_insert': 0.0, 'lambda_sterics_delete': 0.0, 'lambda_electrostatics_insert': 0.0, 'lambda_electrostatics_delete': 0.0, 'lambda_bonds': 0.0, 'lambda_angles': 0.0, 'lambda_torsions': 0.0}
DEBUG:distribution_factories:generation_pdf is None; using pdf state as generator pdf
DEBUG:distribution_factories:initial (importance) work: 0.0 with state reduced potential (15.871405189294213) and generation reduced potential (15.871405189294213)
DEBUG:distribution_factories:successfully updated state, proposal work, and auxiliary work as the negative state reduced potential
DEBUG:openmm_states:successfully updated OpenMMPDFState parameters as follows: {'lambda_sterics_core': 0.0, 'lambda_electrostatics_core': 0.0, 'lambda_sterics_insert': 0.0, 'lambda_sterics_delete': 0.0, 'lambda_electrostatics_insert': 0.0, 'lambda_electrostatics_delete': 0.0

In [7]:
[particle.iteration for particle in particles]

[0, 0, 0]

In [8]:
#main loop
while True:
    #update the iteration
    [particle.update_iteration() for particle in particles]
    
    #compute incremental works
    incremental_works = np.array([target_factory.compute_incremental_work(particle, neglect_proposal_work = True) for particle in particles])
    
    #propagate particles
    [proposal_factory.propagate(particle) for particle in particles]
    
    #resample
    #resampler.resample(particles, incremental_works, observable = nESS, threshold = threshold_function, update_particle_indices = True)
    resampler.resample(particles, incremental_works, None, None)
    
    #ask to terminate
    if all(target_factory.terminate(particle) for particle in particles):
        break
    else:
        #update the auxiliary works...
        [particle.zero_auxiliary_work() for particle in particles]
        [particle.update_auxiliary_work(-target_factory.pdf_state.reduced_potential(particle.state)) for particle in particles]
    

DEBUG:openmm_states:successfully updated OpenMMPDFState parameters as follows: {'lambda_sterics_core': 0.1111111111111111, 'lambda_electrostatics_core': 0.1111111111111111, 'lambda_sterics_insert': 0.2222222222222222, 'lambda_sterics_delete': 0.0, 'lambda_electrostatics_insert': 0.0, 'lambda_electrostatics_delete': 0.2222222222222222, 'lambda_bonds': 0.1111111111111111, 'lambda_angles': 0.1111111111111111, 'lambda_torsions': 0.1111111111111111}
DEBUG:openmm_states:successfully updated OpenMMPDFState parameters as follows: {'lambda_sterics_core': 0.1111111111111111, 'lambda_electrostatics_core': 0.1111111111111111, 'lambda_sterics_insert': 0.2222222222222222, 'lambda_sterics_delete': 0.0, 'lambda_electrostatics_insert': 0.0, 'lambda_electrostatics_delete': 0.2222222222222222, 'lambda_bonds': 0.1111111111111111, 'lambda_angles': 0.1111111111111111, 'lambda_torsions': 0.1111111111111111}
DEBUG:openmm_states:successfully updated OpenMMPDFState parameters as follows: {'lambda_sterics_core':

In [12]:
[sum(particle.incremental_works) for particle in particles]

[1.3375883056206388, 1.3375883056206388, 1.3375883056206388]

seems to work!